In [229]:
%pip install davos
#import davos

#davos.config.suppress_stdout = True

Note: you may need to restart the kernel to use updated packages.


In [230]:
smuggle numpy as np               # pip: numpy==1.24.2
smuggle matplotlib.pyplot as plt  # pip: matplotlib==3.7.0
from matplotlib.patches smuggle Rectangle
from matplotlib.collections smuggle PatchCollection
smuggle pandas as pd              # pip: pandas==1.5.3
smuggle seaborn as sns            # pip: seaborn==0.12.2
from scipy.stats smuggle ttest_rel, ttest_ind, pearsonr      # pip: scipy==1.10.1
from tqdm smuggle tqdm            # pip: tqdm==4.64.1

smuggle requests                  # pip: requests==2.28.2

import os
import warnings

from helpers import load_data, figdir, attention_colors, recency, plot_colorbar

# Download dataset, set up paths, and load behavioral and gaze data into the current workspace:
  - sustained, variable: behavioral data with labels added and all trials removed where participants looked at the attended image (used in most analyses)
  - sustained_gaze, variable_gaze: eyetracker data (used to summarize/plot gaze data)
  - sustained_unfiltered, variable_unfiltered: behavioral data with no trials removed (used to summarize which trials were filtered)

In [ ]:
sustained_gaze, variable_gaze, sustained_unfiltered, variable_unfiltered = load_data()

100%|███| 23/23 [00:03<00:00,  6.49it/s]
2400it [00:05, 476.10it/s]
1840it [00:03, 524.66it/s]
2532it [00:04, 557.72it/s]

In [ ]:
sustained_gaze_orig = sustained_gaze.copy
sustained_gaze_new  = sustained_gaze.drop_duplicates()

sustained_gaze_new

In [ ]:
variable_unfiltered

In [ ]:
variable_gaze_orig = variable_gaze.copy
variable_gaze_new  = variable_gaze.drop_duplicates()

variable_gaze_new

# Now, let's filter the behavioral data

In [ ]:
def filter_data(df, intersection='Attended intersection', min_hertz=20):
    '''
    this function returns filtered behavioral data
    
    it removes :
    - presentation trials where the intersection criteria are not met
    - presentation trials where the minimum amount of gaze data is not recorded
    - memory trials showing images for which intersection criteria were not met 
    - memory trials showing images for which the minimum amount of gaze data was not recorded 
    
    pres options: 'Attended intersection', 'Intersection detected'
    min_hertz: int or float
    of any value
    
    '''
    
    df_list = []
    
    # filter presentation trials based on criteria
    filtered_data_pres = df[  (df['Trial Type'] == 'Presentation') 
                            & (df[intersection] == False)
                            & (df['first_second' ] >= min_hertz)
                            & (df['second_second'] >= min_hertz)
                            & (df['third_second' ] >= min_hertz)
                           ]
    # note: we check the minimum number of gaze datapoints in each second of the 3s presentation trial
    
    # append filtered presentation data to df_list
    df_list.append(filtered_data_pres)
    
    # for each subject
    for s in filtered_data_pres['Subject'].unique():
        
        # get the data for this subject
        sub_df      = df[ df['Subject'] == s ]
        sub_pres_df = filtered_data_pres[filtered_data_pres['Subject'] == s]
    
        # get the images from the trials where this subject had good data
        filtered_pres_images = list(sub_pres_df['Cued Place']) + list(sub_pres_df['Cued Face']) + list(sub_pres_df['Uncued Place']) + list(sub_pres_df['Uncued Face'])

        
        # get the corresponding memory trials
        filtered_data_mem_1    = sub_df[ (sub_df['Trial Type'] == 'Memory')
                                       & (sub_df['Memory Image'].isin(filtered_pres_images))
                                       ]
        
        # also get memory trials with novel images
        filtered_data_mem_2    = sub_df[ (sub_df['Trial Type'] == 'Memory')
                                       & (sub_df['Attention']  == 'Novel')
                                       ]
        
        # append filtered memory data for this subject to the df_list 
        df_list.append( filtered_data_mem_1 )
        df_list.append( filtered_data_mem_2 )
        
    # concatenate all filtered dataframes
    full_filtered = pd.concat(df_list)
    
    
    return( full_filtered )
    

In [ ]:
sustained = filter_data(sustained_unfiltered)
variable  = filter_data(variable_unfiltered)

In [ ]:
sustained.shape

In [ ]:
sustained_unfiltered.shape

In [ ]:
print(len(sustained_unfiltered['Subject'].unique()))
print(len(sustained['Subject'].unique()))

In [ ]:
(12000-8282) / 12000

In [ ]:
variable.shape

In [ ]:
variable_unfiltered.shape

In [ ]:
(9200-5863) / 9200

In [ ]:
print(len(variable_unfiltered['Subject'].unique()))
print(len(variable['Subject'].unique()))

# Figure S1

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(10, 3), sharex=True, sharey=True)

# sns.histplot(sustained_gaze, x='x', y='y', cbar=False, stat='probability', cmap='gray_r', bins=(120, 78), ax=ax[0], vmin=0, vmax=0.03)
# ax[0].text(59.8 / 2, 33.6 / 2, '+', ha='center', va='center', fontsize=10, color='red', fontweight='bold')

# # KZ edited this line
# # im_len = 6.7 * (52.96 / 59.8)
# im_len = 6.7 * (59.8 / 52.96)

# # this line intends to give us the length of an image stimulus in centimeters (instead of DVA)
# # so what (I think?) we want is:
# # im_len_in_cm  =  6.7 DVA  *  ( 59.8 cm  /  52.96 DVA )

# # ^^ I could definitely just be looking at this through tired eyes so it would be great if you could confirm!

# y = (33.6 - im_len) / 2
# x1 = (59.8 / 2) - 4.5 - im_len
# x2 = (59.8 / 2) + 4.5

# images = [Rectangle((x1, y), im_len, im_len, fill=False, color='red', lw=1),
#           Rectangle((x2, y), im_len, im_len, fill=False, color='red', lw=1)]
# pc = PatchCollection(images, match_original=True)
# ax[0].add_collection(pc)

# ax[0].set_xlabel('x (cm)', fontsize=12)
# ax[0].set_ylabel('y (cm)', fontsize=12)
# ax[0].set_title('Sustained', fontsize=12)

# sns.histplot(variable_gaze, x='x', y='y', cbar=False, stat='probability', cmap='gray_r', bins=(120, 78), ax=ax[1], vmin=0, vmax=0.03)
# ax[1].text(59.8 / 2, 33.6 / 2, '+', ha='center', va='center', fontsize=10, color='red', fontweight='bold')
# pc = PatchCollection(images, match_original=True)
# ax[1].add_collection(pc)

# ax[1].set_xlim(0, 59.8)
# ax[1].set_ylim(0, 33.6)
# ax[1].set_xlabel('x (cm)', fontsize=12)
# ax[1].set_ylabel('y (cm)', fontsize=12)
# ax[1].set_title('Variable', fontsize=12)

# plt.subplots_adjust(wspace=0.05)
# fig.savefig(os.path.join(figdir, 'gaze_distribution.pdf'), bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3), sharex=True, sharey=True)

# kz edit: used the gaze data with duplicates dropped
#sns.histplot(sustained_gaze, x='x', y='y', cbar=False, stat='probability', cmap='gray_r', bins=(120, 78), ax=ax[0], vmin=0, vmax=0.03)
sns.histplot(sustained_gaze_new, x='x', y='y', cbar=False, stat='probability', cmap='gray_r', bins=(120, 78), ax=ax[0], vmin=0, vmax=0.03)
ax[0].text(59.8 / 2, 33.6 / 2, '+', ha='center', va='center', fontsize=10, color='red', fontweight='bold')


im_len = 6.7 * ( 59.8 / 52.96 ) # <- kz added this line
# im_len = 6.7 * (52.96 / 59.8) # <- kz commented this line

# KZ NOTE: im_len_in_cm = 6.7 DVA * ( 59.8 cm / 52.96 DVA )
# KZ NOTE: this conversion factor has also been change in the helpers.py

y = (33.6 - im_len) / 2
x1 = (59.8 / 2) - 4.5 - im_len
x2 = (59.8 / 2) + 4.5


# kz added these lines: boundary one cm around the images ##############################
im_len_a = 6.7 * (59.8 / 52.96) + 2
y_a = (33.6 - im_len_a) / 2
x1_a = (59.8 / 2) - 3.5 - im_len_a
x2_a = (59.8 / 2) + 3.5
########################################################################################

images = [Rectangle((x1, y), im_len, im_len, fill=False, color='red', lw=1),
          Rectangle((x2, y), im_len, im_len, fill=False, color='red', lw=1)]
pc = PatchCollection(images, match_original=True)
ax[0].add_collection(pc)


# kz added these lines - draw bounding box around the images ##########################
bounding = [Rectangle((x1_a, y_a), im_len_a, im_len_a, fill=False, color='gray', lw=1),
          Rectangle((x2_a, y_a), im_len_a, im_len_a, fill=False, color='gray', lw=1)]
bounds = PatchCollection(bounding, match_original=True)
ax[0].add_collection(bounds)
########################################################################################

ax[0].set_xlabel('x (cm)', fontsize=12)
ax[0].set_ylabel('y (cm)', fontsize=12)
ax[0].set_title('Sustained', fontsize=12)

# kz edit: used the gaze data with duplicates dropped
# sns.histplot(variable_gaze, x='x', y='y', cbar=False, stat='probability', cmap='gray_r', bins=(120, 78), ax=ax[1], vmin=0, vmax=0.03)
sns.histplot(variable_gaze_new, x='x', y='y', cbar=False, stat='probability', cmap='gray_r', bins=(120, 78), ax=ax[1], vmin=0, vmax=0.03)
ax[1].text(59.8 / 2, 33.6 / 2, '+', ha='center', va='center', fontsize=10, color='red', fontweight='bold')
pc = PatchCollection(images, match_original=True)
bounds = PatchCollection(bounding, match_original=True) # <- kz added this line
ax[1].add_collection(pc)
ax[1].add_collection(bounds) # <- kz added this line

ax[1].set_xlim(0, 59.8)
ax[1].set_ylim(0, 33.6)
ax[1].set_xlabel('x (cm)', fontsize=12)
ax[1].set_ylabel('y (cm)', fontsize=12)
ax[1].set_title('Variable', fontsize=12)

plt.subplots_adjust(wspace=0.05)
fig.savefig(os.path.join(figdir, 'gaze_distribution.pdf'), bbox_inches='tight')

In [ ]:
plot_colorbar('gray_r', 'gray_r')

# Confirm fast reaction time to prode on attended versus unattended side

In [ ]:
def summarize_intersections(df, category='Attended intersection'):
    df = df.query('`Trial Type` == "Presentation"').fillna(False)
    return df[['Subject', 'Run', category]].groupby(['Subject', 'Run']).mean().reset_index().rename({category: 'Intersection'}, axis=1)

In [ ]:
intersection_type = 'Attended intersection'

def plot_intersections(intersection_type, fname=None):
    fig, ax = plt.subplots(2, 2, figsize=(8, 6), sharey=True)
    # sustained
    df = summarize_intersections(sustained_unfiltered, category=intersection_type)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sns.barplot(data=df, x='Run', y='Intersection', ax=ax[0, 0], color='gray')
    ax[0, 0].set_title('Sustained', fontsize=12);
    ax[0, 0].set_ylabel('Proportion of trials', fontsize=12);
    ax[0, 0].set_xlabel('Trial block', fontsize=12);
    sns.despine(ax=ax[0, 0], top=True, right=True)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sns.barplot(data=df, x='Subject', y='Intersection', ax=ax[1, 0], color='gray')
    ax[1, 0].set_ylabel('Proportion of trials', fontsize=12);
    ax[1, 0].set_xlabel('Participant', fontsize=12);
    ax[1, 0].set_xticklabels([i if i % 5 == 0 else '' for i in range(len(df['Subject'].unique()))]);
    sns.despine(ax=ax[1, 0], top=True, right=True)

    # variable
    df = summarize_intersections(variable_unfiltered, category=intersection_type)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sns.barplot(data=df, x='Run', y='Intersection', ax=ax[0, 1], color='gray')
    ax[0, 1].set_title('Variable', fontsize=12);
    ax[0, 1].set_xlabel('Trial block', fontsize=12);
    ax[0, 1].set_ylabel('');
    sns.despine(ax=ax[0, 1], top=True, right=True)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sns.barplot(data=df, x='Subject', y='Intersection', ax=ax[1, 1], color='gray')
    ax[1, 1].set_ylabel('');
    ax[1, 1].set_xlabel('Participant', fontsize=12);
    ax[1, 1].set_xticklabels([i if i % 5 == 0 else '' for i in range(len(df['Subject'].unique()))]);
    sns.despine(ax=ax[1, 1], top=True, right=True)

    ax[0, 0].set_ylim(0, 1.1)
    plt.tight_layout()

    if fname is not None:
        fig.savefig(os.path.join(figdir, fname + '.pdf'), bbox_inches='tight')
    
    return fig

# Figure S2

In [ ]:
plot_intersections(intersection_type, intersection_type.replace(' ', '_'));

In [ ]:
def plot_attention(df, fname=None, palette=attention_colors, ylim=[1.6, 3], **kwargs):
    df = df.copy()

    fig = plt.figure(figsize=(4, 3))
    ax = plt.gca()

    order = ['Attended', 'Attended category', 'Attended location', 'Unattended', 'Novel']

    id_vars = ['Subject', 'Attention', 'Run']
    if 'hue' in kwargs:
        id_vars.append(kwargs['hue'])
    
    sns.barplot(data=df.query('`Trial Type` == "Memory"')[[*id_vars, 'Familiarity Rating']].groupby(id_vars).mean().reset_index(), 
                   x='Attention', y='Familiarity Rating', order=['Attended', 'Attended category', 'Attended location', 'Unattended', 'Novel'],  palette=palette, **kwargs);
    ax.set_xlabel('Attention level', fontsize=12);
    ax.set_ylabel('Familiarity rating', fontsize=12);
    ax.set_xticklabels(['' for _ in range(len(ax.get_xticklabels()))]);
    ax.set_ylim(ylim)

    if 'hue' in kwargs:                
        n = len(df[kwargs['hue']].unique()) - 1  # not sure why this correction is needed...
        alphas = np.linspace(1, 0, n + 1)[:-1]

        for i in range(n):
            for j, bar in enumerate(ax.containers[i]):
                bar.set_color(palette[order[j]])
                bar.set_alpha(alphas[i])

    sns.despine(top=True, right=True)

    legend = ax.get_legend()
    if legend is not None:
        legend.remove()

    if fname is not None:
        fig.savefig(os.path.join(figdir, fname + '.pdf'), bbox_inches='tight')
    
    return fig

# Figures 2 and S3

In [ ]:
plot_attention(sustained, fname='sustained_attention');

In [ ]:
#plot_attention(sustained_unfiltered, fname='sustained_attention');

In [ ]:
plot_attention(sustained, fname='sustained_attention_by_category', hue='Category');

In [ ]:
#plot_attention(sustained_unfiltered, fname='sustained_attention_by_category', hue='Category')

In [ ]:
plot_attention(variable, fname='variable_attention');

In [ ]:
plot_attention(variable, fname='variable_attention_by_category', hue='Category');

# Stats!

Within-condition comparisons by attention level

In [ ]:
def ttests_by_attention_level(df, category=None):
    def print_ttest_results(results, label1, label2):
        if results.pvalue < 0.001:
            p_string = 'p < 0.001'
        else:
            p_string = f'p = {results.pvalue:.3f}'

        print(f'{label1} vs. {label2}: $t({results.df}) = {results.statistic:.3f}, {p_string}$')

    if category is not None:
        if type(category) is str:
            category = [category]
        for c in category:
            print(f'\nCategory: {c}')
            ttests_by_attention_level(df.query('Category == @c'))

        print('\n\Within-level tests:')
        df = df.groupby(['Subject', 'Attention', 'Category']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index=['Subject', 'Category'], columns='Attention', values='Familiarity Rating').reset_index().set_index('Subject')

        for c in df.columns[1:]:
            for i, c1 in enumerate(category):
                for c2 in category[i + 1:]:
                    print_ttest_results(ttest_rel(df.loc[df['Category'] == c1, c], df.loc[df['Category'] == c2, c]), f'{c} {c1}', c2)

        return
    
    # re-organize dataframe-- rows: subjects; columns: attention levels
    df = df.groupby(['Subject', 'Attention']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index='Subject', columns='Attention', values='Familiarity Rating')

    # run t-tests
    for i, c1 in enumerate(df.columns):
        for c2 in df.columns[i + 1:]:
            print_ttest_results(ttest_rel(df[c1], df[c2]), c1, c2)            
    
    print('\n')

    # old vs. new
    print_ttest_results(ttest_rel(df[['Attended', 'Attended category', 'Attended location', 'Unattended']].mean(axis=1), df['Novel']), 'Old', 'New')

    # location benefit
    print_ttest_results(ttest_rel(df[['Attended', 'Attended location']].mean(axis=1), df['Novel']), 'Attended + Attended location', 'Novel')

    # category benefit
    print_ttest_results(ttest_rel(df[['Attended', 'Attended category']].mean(axis=1), df['Novel']), 'Attended + Attended category', 'Novel')

In [ ]:
ttests_by_attention_level(sustained)

In [ ]:
ttests_by_attention_level(sustained, category=['Face', 'Place'])

In [ ]:
ttests_by_attention_level(variable)

In [ ]:
ttests_by_attention_level(variable, category=['Face', 'Place'])

Across-condition comparisons

In [ ]:
def across_condition_ttests_by_attention_level(df1, df2, category=None, names=['Sustained', 'Variable']):
    def print_ttest_results(a, b, prefix):
        results = ttest_ind(a, b)
        if results.pvalue < 0.001:
            p_string = 'p < 0.001'
        else:
            p_string = f'p = {results.pvalue:.3f}'

        df = len(a) + len(b) - 2
        label1 = f'{prefix} -- {names[0]}'
        label2 = names[1]
        print(f'{label1} vs. {label2}: $t({df}) = {results.statistic:.3f}, {p_string}$')

    if category is not None:
        if type(category) is str:
            category = [category]
        for c in category:
            print(f'\nCategory: {c}')
            across_condition_ttests_by_attention_level(df1.query('Category == @c'), df2.query('Category == @c'), names=names)
        
        print('\nWithin-level tests:')
        df1 = df1.groupby(['Subject', 'Attention', 'Category']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index=['Subject', 'Category'], columns='Attention', values='Familiarity Rating').reset_index().set_index('Subject')
        df2 = df2.groupby(['Subject', 'Attention', 'Category']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index=['Subject', 'Category'], columns='Attention', values='Familiarity Rating').reset_index().set_index('Subject')

        for attention in df1.columns[1:]:
            for cat in category:
                print_ttest_results(df1.loc[df1['Category'] == cat, attention], df2.loc[df2['Category'] == cat, attention], f'{attention} {cat}')

        return
    
    # re-organize dataframe-- rows: subjects; columns: attention levels
    df1 = df1.groupby(['Subject', 'Attention']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index='Subject', columns='Attention', values='Familiarity Rating')
    df2 = df2.groupby(['Subject', 'Attention']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index='Subject', columns='Attention', values='Familiarity Rating')

    # run t-tests
    for attention in df1.columns:
        print_ttest_results(df1[attention], df2[attention], attention)
    
    print('\n')

    # old
    print_ttest_results(df1[['Attended', 'Attended category', 'Attended location', 'Unattended']].mean(axis=1), df2[['Attended', 'Attended category', 'Attended location', 'Unattended']].mean(axis=1), 'Old')

    # location benefit
    print_ttest_results(df1[['Attended', 'Attended location']].mean(axis=1), df2[['Attended', 'Attended location']].mean(axis=1), 'Attended + Attended location')

    # category benefit
    print_ttest_results(df1[['Attended', 'Attended category']].mean(axis=1), df2[['Attended', 'Attended category']].mean(axis=1), 'Attended + Attended category')

In [ ]:
across_condition_ttests_by_attention_level(sustained, variable)

In [ ]:
across_condition_ttests_by_attention_level(sustained, variable, category=['Face', 'Place'])

Face vs. place differences by condition

In [ ]:
def face_vs_place_ttest(df):
    df = df.groupby(['Subject', 'Category']).mean(numeric_only=True)['Familiarity Rating'].reset_index().pivot(index='Subject', columns='Category', values='Familiarity Rating')
    results = ttest_rel(df['Place'], df['Face'])
    if results.pvalue < 0.001:
        p_string = 'p < 0.001'
    else:
        p_string = f'p = {results.pvalue:.3f}'
    print(f'$t({results.df}) = {results.statistic:.3f}, {p_string}$')

In [ ]:
print('Sustained condition: place vs. face familiarity')
face_vs_place_ttest(sustained)

In [ ]:
print('Variable condition: place vs. face familiarity')
face_vs_place_ttest(variable)

## Serial position effects during *encoding*

Plot familiarity as a function of presentation position:
  - $x$-axis: study position
  - $y$-axis: familiarity (at recall)
  - color: attention level

In [ ]:
def encoding_df(df):
    df = df.query('`Trial Type` == "Presentation"')[['Subject', 'Run', 'Order', 'Attended', 'Attended category', 'Attended location', 'Unattended', 'Cued Location', 'Cued Category']]
    df = df.drop('Run', axis=1).groupby(['Subject', 'Order']).mean(numeric_only=True).reset_index()
    return df.melt(id_vars=['Subject', 'Order'], value_vars=['Attended', 'Attended category', 'Attended location', 'Unattended'], var_name='Attention', value_name='Familiarity Rating')

# Figure 4A and B

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 3), sharey=True, sharex=True)
sns.lineplot(encoding_df(sustained), x='Order', y='Familiarity Rating', hue='Attention', ax=ax[0], legend=False, palette=attention_colors)
ax[0].set_xlabel('Presentation position', fontsize=12)
ax[0].set_ylabel('Familiarity rating', fontsize=12)
ax[0].set_title('Sustained', fontsize=12)
sns.despine(top=True, right=True)

sns.lineplot(encoding_df(variable), x='Order', y='Familiarity Rating', hue='Attention', ax=ax[1], legend=False, palette=attention_colors)
ax[1].set_xlabel('Presentation position', fontsize=12)
ax[1].set_ylabel('')
ax[1].set_title('Variable', fontsize=12)
sns.despine(top=True, right=True)

ax[0].set_xlim([0, 9])
ax[0].set_ylim([1, 3.5])
plt.tight_layout()

fig.savefig(os.path.join(figdir, 'encoding_effects.pdf'), bbox_inches='tight')

Plot familiarity as a function of recall position:
  - $x$-axis: recall position
  - $y$-axis: familiarity
  - color: attention level

# Figure 4C and D

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 3), sharey=True, sharex=True)

sns.lineplot(sustained.query('`Trial Type` == "Memory"'), x='Order', y='Familiarity Rating', hue='Attention', palette=attention_colors, legend=False, ax=ax[0])
ax[0].set_xlabel('Probe position', fontsize=12)
ax[0].set_ylabel('Familiarity rating', fontsize=12)
ax[0].set_title('Sustained', fontsize=12)
sns.despine(top=True, right=True)

sns.lineplot(variable.query('`Trial Type` == "Memory"'), x='Order', y='Familiarity Rating', hue='Attention', palette=attention_colors, legend=False, ax=ax[1])
ax[1].set_xlabel('Probe position', fontsize=12)
ax[1].set_ylabel('')
ax[1].set_title('Variable', fontsize=12)
sns.despine(top=True, right=True)

ax[0].set_xlim([0, 39])
ax[0].set_ylim([1, 3.5])
plt.tight_layout()

fig.savefig(os.path.join(figdir, 'familiarity_by_probe_position.pdf'), bbox_inches='tight')



In [ ]:
def cue_effect_heatmaps(df, fname=None):

    attention_levels = ['Attended', 'Attended category', 'Attended location', 'Unattended']

    cue_matches = ['Same cue sequence length', 'Category sequence length', 'Location sequence length']

    fig, ax = plt.subplots(nrows=len(cue_matches), ncols=len(attention_levels), figsize=(12, 9), sharey=True, sharex='row')
    x = df.query('`Trial Type` == "Presentation"')

    for j, c in enumerate(cue_matches):
        for i, a in enumerate(attention_levels):            
            sns.histplot(data=x, x=c, y=a, discrete=True, cmap=sns.light_palette(attention_colors[a], as_cmap=True), stat='probability', common_norm=True, cbar=False, ax=ax[j, i], vmin=0, vmax=0.05)
            sns.regplot(data=x, x=c, y=a, color='k', scatter=False, ax=ax[j, i])
            ax[j, i].set_xlabel(c, fontsize=12)

            if j == 0:
                ax[j, i].set_title(a, fontsize=14)
            if i == 0:
                ax[j, i].set_ylabel('Familiarity rating', fontsize=12)
            else:
                ax[j, i].set_ylabel('')
            
            ax[j, i].set_yticks([1, 2, 3, 4])
            
            ax[j, i].set_xlim([0.5, x[c].max() + 0.5])
            ax[j, i].set_ylim([0.5, 4.5])

            vals = x[[a, c]].dropna(how='any', axis=0)
            r = pearsonr(vals[c], vals[a])
            if r.pvalue < 0.001:
                p_string = 'p < 0.001'
            else:
                p_string = f'p = {r.pvalue:.3f}'

            print(f'{a} {c} $r = {r.statistic:.3f}, {p_string}$')

    plt.tight_layout()

    if fname is not None:
        fig.savefig(os.path.join(figdir, fname + '.pdf'), bbox_inches='tight')
    
    return fig

# Figure S4

In [ ]:
cue_effect_heatmaps(sustained, fname='sustained_cue_effects');

In [ ]:
for k, v in attention_colors.items():
    plot_colorbar(sns.light_palette(v, as_cmap=True), f'{k.lower()}_light_colorbar')

In [ ]:
cue_effect_heatmaps(variable, fname='variable_cue_effects');

Recency and response biases:
  - For each probe, compute the temporal distance (in image presentations) to the nearest same-category cue
  - For each probe, compute the number of same-category cues from the current run
  - For each probe, compute a recency-weighted average of the number of same-category cues on most recent run, where

$w = \argmax\left[1 - \exp\{-\frac{x}{\tau}\} , \epsilon \right]$,

and where $w$ is the weight given to the cue at presentation position $x$, $\tau = 2$, and $\epsilon = 0.05$.

# Figure S5C

In [ ]:
fig = plt.figure(figsize=(4, 2.8))

x = np.linspace(0, 9, 10)
plt.plot(x, recency(x, tau=2, eps=0.05) / sum(recency(x, tau=2, eps=0.05)), 'ko-')
plt.xlabel('Presentation position', fontsize=12)
plt.ylabel('Recency weight', fontsize=12)
plt.ylim([0, 0.14])
plt.xlim([-0.3, 9.3])
sns.despine(top=True, right=True)

fig.savefig(os.path.join(figdir, 'recency_weights.pdf'), bbox_inches='tight')

# Figure S5A

In [ ]:
# fig = plt.figure(figsize=(4, 2.8))
# sns.barplot(sustained.query('`Trial Type` == "Memory" and Attention == "Novel"'), x='Category-matched recent cue', y='Familiarity Rating', color=attention_colors['Novel'])
# plt.xlabel('Category-matched recent cue', fontsize=12)
# plt.ylabel('Familiarity rating', fontsize=12)
# plt.ylim([1.6, 3])
# plt.tight_layout()
# sns.despine(top=True, right=True)

# # statistical test
# vals = sustained.query('`Trial Type` == "Memory" and Attention == "Novel"').groupby(['Subject', 'Category-matched recent cue']).mean(numeric_only=True).reset_index().set_index('Subject')[['Category-matched recent cue', 'Familiarity Rating']].dropna(how='any', axis=0)

# fig.savefig(os.path.join(figdir, 'sustained_category_match_recent_cue.pdf'), bbox_inches='tight')

Look for familiarity differences (in the sustained condition) between attended vs. unattended category novel images:

In [ ]:
# result = ttest_rel(vals.query('`Category-matched recent cue` == True')['Familiarity Rating'], vals.query('`Category-matched recent cue` == False')['Familiarity Rating'])
# if result.pvalue < 0.001:
#     p_string = 'p < 0.001'
# else:
#     p_string = f'p = {result.pvalue:.3f}'
# print(f'Response bias (increase in familiarity for novel images from the most recently cued category): $t({result.df}) = {result.statistic:.3f}, {p_string}$')

Look for familiarity differences (in the sustained condition) between attended category *targets* vs. attended category *lures*:

In [ ]:
# attended_category_targets = sustained.query('`Trial Type` == "Memory" and Attention == "Attended category"').groupby(['Subject', 'Category-matched recent cue']).mean(numeric_only=True).reset_index().set_index('Subject')[['Familiarity Rating']].dropna(how='any', axis=0)
# attended_category_lures = sustained.query('`Trial Type` == "Memory" and Attention == "Novel" and `Category-matched recent cue`').groupby(['Subject']).mean(numeric_only=True).reset_index().set_index('Subject')[['Familiarity Rating']].dropna(how='any', axis=0)

# result = ttest_rel(attended_category_targets['Familiarity Rating'], attended_category_lures['Familiarity Rating'])
# if result.pvalue < 0.001:
#     p_string = 'p < 0.001'
# else:
#     p_string = f'p = {result.pvalue:.3f}'
# print(f'Familiarity for attended category targets vs lures: $t({result.df}) = {result.statistic:.3f}, {p_string}$')

In [ ]:
def cue_recency_heatmaps(df, metrics=None, fname=None):

    attention_levels = ['Attended', 'Attended category', 'Attended location', 'Unattended', 'Novel']

    if metrics is None:
        metrics = ['Distance to nearest same-category cue', 'Recency-weighted number of same-category cues']

    fig, ax = plt.subplots(nrows=len(metrics), ncols=len(attention_levels), figsize=(15, 6), sharey=True, sharex='row')
    x = df.query('`Trial Type` == "Memory"')

    for j, c in enumerate(metrics):
        for i, a in enumerate(attention_levels):
            xa = x.query('Attention == @a').sort_values(by=[c])

            if xa.shape[0] == 0:
                ax[j, i].set_axis_off()
                continue

            if c == 'Recency-weighted number of same-category cues':
                bins = np.histogram(xa[c], bins=25)[1]
                xa['Digitized recency-weighted number of same-category cues'] = np.digitize(xa[c], bins)
                cx = 'Digitized recency-weighted number of same-category cues'
                vmax = 0.025
            else:
                vmax = 0.15
                cx = c


            sns.histplot(data=xa, x=cx, y='Familiarity Rating', discrete=True, cmap=sns.light_palette(attention_colors[a], as_cmap=True), stat='probability', vmin=0, vmax=vmax, common_norm=True, cbar=False, ax=ax[j, i])
            sns.regplot(data=xa, x=cx, y='Familiarity Rating', color='k', scatter=False, ax=ax[j, i])
            ax[j, i].set_xlabel(c.replace('same-', '\nsame-'), fontsize=12)

            if j == 0:
                ax[j, i].set_title(a, fontsize=14)
            if i == 0:
                ax[j, i].set_ylabel('Familiarity rating', fontsize=12)
            else:
                ax[j, i].set_ylabel('')
            
            ax[j, i].set_yticks([1, 2, 3, 4])

            ax[j, i].set_xlim([0.5, xa[cx].max() + 0.5])
            ax[j, i].set_ylim([0.5, 4.5])                                                

            vals = xa[['Familiarity Rating', c]].dropna(how='any', axis=0)
            r = pearsonr(vals[c], vals['Familiarity Rating'])
            if r.pvalue < 0.001:
                p_string = 'p < 0.001'
            else:
                p_string = f'p = {r.pvalue:.3f}'

            print(f'{a} {c} $r = {r.statistic:.3f}, {p_string}$')

    plt.tight_layout()

    # fix up recency-weighted labels
    for j, c in enumerate(metrics):
        if c == 'Recency-weighted number of same-category cues':
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")  # ignore FixedLocator warning
                ax[j, 0].set_xticklabels([f'{bins[int(i) - 1] + bins[int(i)] / 2:0.2f}' if (i > 1 and i < len(bins)) else '' for i in ax[j, 0].get_xticks()])

    if fname is not None:
        fig.savefig(os.path.join(figdir, fname + '.pdf'), bbox_inches='tight')
    
    return fig

# Figure S5B

In [ ]:
cue_recency_heatmaps(variable, fname='cue_recency_heatmaps_variable');